_________________________________________________

### FEATURE ENGINEERING

In [693]:
import pandas as pd
clean_df= pd.read_csv('part_1.csv')
clean_df.drop('Unnamed: 0',inplace = True,axis = 1)
clean_df.head(3)

,screen_name,carrier,location,date_time,text,clean_text,polarity,fact,month,hour,sense_dummy,text_len,day,day_of_month,part_of_day,seconds,sin_time,cos_time,tokens,topic
0,TheSkubis,verizonsupport,"Pennsylvania, USA",2021-03-30 23:59:19,@VerizonSupport I have issues with closed capt...,issue closed caption walking dead show display...,-0.150,1,3,23,-1,51,1,30,evening,86359,-0.002982,0.999996,"['issue', 'closed', 'caption', 'walking', 'dea...",4
1,VerizonSupport,verizonsupport,NaN,2021-03-30 23:56:55,@_carolinek This could be due to regional rest...,this could due regional restriction have check...,-0.125,1,3,23,-1,52,1,30,evening,86215,-0.013453,0.999910,"['this', 'could', 'due', 'regional', 'restrict...",12
2,VerizonSupport,verizonsupport,NaN,2021-03-30 23:55:30,@tvmurray We'll be happy to help with anything...,happy help anything related fios twitter pleas...,0.400,0,3,23,1,63,1,30,evening,86130,-0.019634,0.999807,"['happy', 'help', 'anything', 'related', 'fios...",13


In [694]:
# убираем пустые строки
clean_df = clean_df[~clean_df.clean_text.isnull()]
clean_df.shape

(638, 20)

In [695]:
# 1. drop duplicates

clean_df = clean_df.drop_duplicates('clean_text')
clean_df.shape

(531, 20)

In [696]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# благодаря параметру max_features мы оставляем 80% самых выжных фичей
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2),max_features = int(0.8 * 4309))
final_tf_idf = tf_idf_vect.fit_transform(clean_df['clean_text'].values)

final_tf_idf.shape

(531, 3447)

In [697]:
# начинаем создавать матрицу фичей

# добавление tfidf в нашу матрицу фичей 
feature_names = tf_idf_vect.get_feature_names()
feature_df = pd.DataFrame(final_tf_idf.toarray(), columns = feature_names)
feature_df.shape

(531, 3447)

In [698]:
# clean_df['clean_text'][0]

In [699]:
# кумулятивный вес слова - оставить только те слова, которые суммарно приносят 80% идф
# accum = feature_df.transpose()
# words = [] 
# accum = accum[accum[0] !=0]
# accum
# accum['%'] = accum[0]/accum[0].sum()
# start = 0
# accum['plus'] = accum['%'].cumsum()
# accum = accum[accum['plus']<0.8]
# accum

In [700]:
# вероятность городов
def city_proba(city):
    if type(city) != str:
        return clean_df[clean_df['location'].isna()]['location'].shape[0]/clean_df.shape[0]
    else:
        proba = clean_df.location.value_counts()/clean_df.shape[0]
    return proba[city]

city_proba('Stockholm, Sweden')


0.03766478342749529

In [701]:
# проверка количества строк
# clean_df[clean_df['location'].isna()]['location'].shape[0]+clean_df.location.value_counts().sum()

In [702]:
# добавляем вероятность городов в clean_df и в матрицу фичей
clean_df['location_proba'] = clean_df['location'].apply(lambda x:city_proba(x))
location_proba = list(clean_df['location_proba'])
feature_df['location_proba'] = location_proba



In [703]:
feature_df[feature_df['location_proba'].isna()].shape

(0, 3448)

### RFM analysis

In [704]:
clean_df.head(3)

,screen_name,carrier,location,date_time,text,clean_text,polarity,fact,month,hour,...,text_len,day,day_of_month,part_of_day,seconds,sin_time,cos_time,tokens,topic,location_proba
0,TheSkubis,verizonsupport,"Pennsylvania, USA",2021-03-30 23:59:19,@VerizonSupport I have issues with closed capt...,issue closed caption walking dead show display...,-0.150,1,3,23,...,51,1,30,evening,86359,-0.002982,0.999996,"['issue', 'closed', 'caption', 'walking', 'dea...",4,0.003766
1,VerizonSupport,verizonsupport,NaN,2021-03-30 23:56:55,@_carolinek This could be due to regional rest...,this could due regional restriction have check...,-0.125,1,3,23,...,52,1,30,evening,86215,-0.013453,0.999910,"['this', 'could', 'due', 'regional', 'restrict...",12,0.425612
2,VerizonSupport,verizonsupport,NaN,2021-03-30 23:55:30,@tvmurray We'll be happy to help with anything...,happy help anything related fios twitter pleas...,0.400,0,3,23,...,63,1,30,evening,86130,-0.019634,0.999807,"['happy', 'help', 'anything', 'related', 'fios...",13,0.425612


In [705]:
feature_df.head(3)

,able,able get,able help,able redeem,able watch,absolutely,absolutely horrendous,absolutely suck,acce,access,...,youtube,youtube where,ypr,ypr yard,yr,yr ago,zip,zip code,𝐋𝐄𝐀𝐃𝐄𝐑,location_proba
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003766
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.425612
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.425612


In [706]:
import datetime as dt
import re
clean_df['date'] = clean_df['date_time'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
clean_df['date'] = clean_df['date'].apply(dt.datetime.date)
clean_df.head(1)

,screen_name,carrier,location,date_time,text,clean_text,polarity,fact,month,hour,...,day,day_of_month,part_of_day,seconds,sin_time,cos_time,tokens,topic,location_proba,date
0,TheSkubis,verizonsupport,"Pennsylvania, USA",2021-03-30 23:59:19,@VerizonSupport I have issues with closed capt...,issue closed caption walking dead show display...,-0.15,1,3,23,...,1,30,evening,86359,-0.002982,0.999996,"['issue', 'closed', 'caption', 'walking', 'dea...",4,0.003766,2021-03-30


In [707]:
recency = list(clean_df['date'].apply(lambda x: dt.date.today() - x))
recency
feature_df['recency'] = recency
feature_df['recency'] = feature_df['recency'].apply(lambda x: int(str(x).split()[0]))
feature_df[feature_df['recency'] == 'NaT']

,able,able get,able help,able redeem,able watch,absolutely,absolutely horrendous,absolutely suck,acce,access,...,youtube where,ypr,ypr yard,yr,yr ago,zip,zip code,𝐋𝐄𝐀𝐃𝐄𝐑,location_proba,recency


In [708]:
# feature_df[feature_df['location_proba'].isna()]

In [709]:
# feature_df['frequency'] = f
frequency = list(clean_df['screen_name'].apply(lambda x: clean_df[clean_df['screen_name'] == x].shape[0]))
feature_df['frequency'] = frequency
clean_df['frequency'] = clean_df['screen_name'].apply(lambda x: clean_df[clean_df['screen_name'] == x].shape[0])

In [710]:
monetary = list(clean_df['polarity'])
feature_df['monetary'] = monetary
feature_df.head(3)

,able,able get,able help,able redeem,able watch,absolutely,absolutely horrendous,absolutely suck,acce,access,...,ypr yard,yr,yr ago,zip,zip code,𝐋𝐄𝐀𝐃𝐄𝐑,location_proba,recency,frequency,monetary
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.003766,19,1,-0.150
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.425612,19,37,-0.125
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.425612,19,37,0.400


In [711]:
# recency
r_q3 = feature_df['recency'].quantile(0.3)
r_q6 = feature_df['recency'].quantile(0.6)

feature_df['r'] = feature_df['recency'].apply(lambda x: 1 if x < r_q3 else(2 if r_q3<x<r_q6 else 3)  )
# frequency
f_q3 = feature_df['frequency'].quantile(0.3)
f_q6 = feature_df['frequency'].quantile(0.6)

feature_df['f'] = feature_df['frequency'].apply(lambda x: 1 if x < f_q3 else(2 if f_q3<x<f_q6 else 3)  )

# monetary
m_q3 = feature_df['monetary'].quantile(0.3)
m_q6 = feature_df['monetary'].quantile(0.6)

feature_df['m'] = feature_df['monetary'].apply(lambda x: 1 if x < m_q3 else(2 if m_q3<x<m_q6 else 3)  )
feature_df['rfm'] = feature_df['r']*100 + feature_df['f']*10 + feature_df['m']

In [712]:
feature_df.drop(['r','f','m'],inplace = True,axis = 1)
feature_df.drop(['recency','frequency','monetary'],inplace = True,axis = 1)
feature_df.head(3)

,able,able get,able help,able redeem,able watch,absolutely,absolutely horrendous,absolutely suck,acce,access,...,youtube where,ypr,ypr yard,yr,yr ago,zip,zip code,𝐋𝐄𝐀𝐃𝐄𝐑,location_proba,rfm
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003766,331
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.425612,331
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.425612,333


In [713]:
# добавим метрики, которые можно было бы использовать для кластеризации
sin  = list(clean_df['sin_time'])
cos = list(clean_df['cos_time'])
text_len  = list(clean_df['text_len'])
topic = list(clean_df['topic'])
fact = list(clean_df['fact'])

feature_df['sin_time'] = sin
feature_df['cos_time'] = cos
feature_df['text_len'] = text_len
feature_df['topic'] = topic
feature_df['fact'] = fact

In [714]:
polarity  = list(clean_df['polarity'])
feature_df['polarity'] = polarity            

In [715]:
# наша матрица фичей
feature_df.head()

,able,able get,able help,able redeem,able watch,absolutely,absolutely horrendous,absolutely suck,acce,access,...,zip code,𝐋𝐄𝐀𝐃𝐄𝐑,location_proba,rfm,sin_time,cos_time,text_len,topic,fact,polarity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.003766,331,-0.002982,0.999996,51,4,1,-0.150
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.425612,331,-0.013453,0.999910,52,12,1,-0.125
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.425612,333,-0.019634,0.999807,63,13,0,0.400
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.425612,333,-0.020070,0.999799,50,11,1,0.000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.425612,333,-0.024287,0.999705,71,11,0,0.450


In [716]:
clean_df.head(1)

,screen_name,carrier,location,date_time,text,clean_text,polarity,fact,month,hour,...,day_of_month,part_of_day,seconds,sin_time,cos_time,tokens,topic,location_proba,date,frequency
0,TheSkubis,verizonsupport,"Pennsylvania, USA",2021-03-30 23:59:19,@VerizonSupport I have issues with closed capt...,issue closed caption walking dead show display...,-0.15,1,3,23,...,30,evening,86359,-0.002982,0.999996,"['issue', 'closed', 'caption', 'walking', 'dea...",4,0.003766,2021-03-30,1


### MIN-MAX scaler

In [717]:
feature_df.describe()

,able,able get,able help,able redeem,able watch,absolutely,absolutely horrendous,absolutely suck,acce,access,...,zip code,𝐋𝐄𝐀𝐃𝐄𝐑,location_proba,rfm,sin_time,cos_time,text_len,topic,fact,polarity
count,531.000000,531.000000,531.000000,531.000000,531.000000,531.000000,531.000000,531.000000,531.000000,531.000000,...,531.000000,531.000000,531.000000,531.000000,531.000000,531.000000,531.000000,531.000000,531.000000,531.000000
mean,0.002444,0.000552,0.000642,0.001145,0.000519,0.000931,0.000409,0.000583,0.000475,0.000526,...,0.002572,0.001883,0.195513,331.227872,-0.282675,0.915731,47.190207,8.926554,0.670433,0.112006
std,0.025176,0.012725,0.014796,0.018667,0.011949,0.015395,0.009427,0.013444,0.010939,0.012132,...,0.027611,0.043396,0.200473,3.605401,0.256351,0.126375,19.646296,5.237768,0.470499,0.301070
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.001883,321.000000,-0.889017,0.457874,3.000000,0.000000,0.000000,-1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.003766,331.000000,-0.450972,0.892531,37.000000,4.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.105461,333.000000,-0.199653,0.979867,50.000000,9.000000,1.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.425612,333.000000,-0.054515,0.998513,61.000000,13.000000,1.000000,0.284524
max,0.284049,0.293239,0.340943,0.319377,0.275343,0.290725,0.217222,0.309806,0.252077,0.279560,...,0.433153,1.000000,0.425612,333.000000,-0.000218,1.000000,106.000000,19.000000,1.000000,1.000000


In [718]:
from sklearn.preprocessing import MinMaxScaler  # класс для скалирования! 

# Он сам будет считать максимумы, минимумы и делить! 

# Объеявили скалировшик 
scaler = MinMaxScaler( )

# Обучили его на тренировочной выборке (он само посчитал все максимумы, минимумы, поделил)
scaler.fit(feature_df) 

# Теперь применяем его к нашим данным 
norm_feature_df = pd.DataFrame(scaler.transform(feature_df),columns =list(feature_df.columns) )
norm_feature_df.shape

(531, 3455)

In [719]:
feature_df_norm

,able,able get,able help,able redeem,able watch,absolutely,absolutely horrendous,absolutely suck,acce,access,...,zip,zip code,𝐋𝐄𝐀𝐃𝐄𝐑,location_proba,rfm,sin_time,cos_time,text_len,topic,fact
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.004444,0.833333,0.996891,0.999992,0.466019,0.736842,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.000000,0.833333,0.985109,0.999833,0.475728,0.000000,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.000000,1.000000,0.978155,0.999644,0.582524,0.052632,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.000000,1.000000,0.977664,0.999629,0.456311,0.842105,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.000000,1.000000,0.972920,0.999456,0.660194,0.052632,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.000000,1.000000,0.620667,0.891856,0.514563,0.947368,1.0
527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,1.000000,0.619204,0.890995,0.213592,0.947368,1.0
528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.244444,1.000000,0.615511,0.888804,0.456311,0.684211,0.0
529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.244444,1.000000,0.612514,0.887008,0.475728,0.210526,1.0


In [720]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# feature_df_norm = scaler.fit_transform(feature_df)

# # statistics of scaled data
# pd.DataFrame(feature_df_norm).describe()

### CLUSTERING

### 1. добавить метрики кластеризации

    https://nafi.ru/upload/spss/Lection_9.pdf
    https://scikit-learn.org/stable/modules/clustering.html
    https://habr.com/ru/company/ods/blog/325654/
    https://habr.com/ru/company/jetinfosystems/blog/467745/
    https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-k-means-clustering/
    https://habr.com/ru/post/67078/
    https://iq.opengenus.org/euclidean-vs-manhattan-vs-chebyshev-distance/#:~:text=the%20distance%20between%20squares%20on,its%20diagonals%20as%20coordinate%20axes
    https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score
    
    

### здесь только fit?

In [721]:
# K-means eucledean distance

num_clusters = 2

# Метод к-средних - KMeans
from sklearn.cluster import KMeans

euclidean_churn = KMeans(n_clusters=num_clusters)

# get_ipython().magic('time km.fit(tfidf_matrix)')
idx = euclidean_churn.fit(feature_df)
# print(km.labels_)
clean_df['euclidean_churn'] = euclidean_churn.labels_
# clusters = km.labels_.tolist()
clean_df['euclidean_churn'].value_counts()

1    371
0    160
Name: euclidean_churn, dtype: int64

In [722]:
silhouettes = []
from sklearn import metrics
silhouette1 = metrics.silhouette_score(feature_df, euclidean_churn.labels_,metric="sqeuclidean")
print(silhouette1)
silhouettes.append(silhouette1)

0.7103709501472437


In [723]:
# pip install --user scikit-learn-extra

In [724]:
# k-medoids Чебышев
from sklearn_extra.cluster import KMedoids

num_clusters = 2
chebyshev_churn = KMedoids(n_clusters=num_clusters,metric = 'chebyshev')
# print(km)

idx = chebyshev_churn.fit(feature_df)
# print(km.labels_)
clean_df['chebyshev_churn'] = chebyshev_churn.labels_
# clusters = km.labels_.tolist()
clean_df['chebyshev_churn'].value_counts()

0    530
1      1
Name: chebyshev_churn, dtype: int64

In [725]:

from sklearn import metrics
silhouette2 = metrics.silhouette_score(feature_df, chebyshev_churn.labels_,metric="sqeuclidean")
print(silhouette2)
silhouettes.append(silhouette2)

-0.6312623168813967


In [726]:
# k-medoids squared eucledean
from sklearn_extra.cluster import KMedoids

num_clusters = 2
sqeuclidean = KMedoids(n_clusters=num_clusters,metric = 'sqeuclidean')
# print(km)

idx = sqeuclidean.fit(feature_df)
# print(km.labels_)
clean_df['sqeuclidean_churn'] = sqeuclidean.labels_
# clusters = km.labels_.tolist()
clean_df['sqeuclidean_churn'].value_counts()

1    343
0    188
Name: sqeuclidean_churn, dtype: int64

In [727]:
from sklearn import metrics
silhouette3= metrics.silhouette_score(feature_df, sqeuclidean.labels_,metric="sqeuclidean")
print(silhouette3)
silhouettes.append(silhouette3)

0.6848596704884136


In [728]:
# Попробуем выделить лейблы churn по очень негативной полярности твитов 
# для дальнейшего сравнения близости выделенных нами групп с искуственно созданным таргетом

clean_df['naive_churn'] = clean_df['polarity'].apply(lambda x:1 if x <-0.5 else 0 )
clean_df['naive_churn'].value_counts()

0    522
1      9
Name: naive_churn, dtype: int64

In [729]:
clean_df.head(1)

,screen_name,carrier,location,date_time,text,clean_text,polarity,fact,month,hour,...,cos_time,tokens,topic,location_proba,date,frequency,euclidean_churn,chebyshev_churn,sqeuclidean_churn,naive_churn
0,TheSkubis,verizonsupport,"Pennsylvania, USA",2021-03-30 23:59:19,@VerizonSupport I have issues with closed capt...,issue closed caption walking dead show display...,-0.15,1,3,23,...,0.999996,"['issue', 'closed', 'caption', 'walking', 'dea...",4,0.003766,2021-03-30,1,1,0,1,0


In [730]:
from sklearn.metrics import roc_auc_score,precision_recall_curve,roc_curve,accuracy_score,recall_score
roc_aucs = []
accuracy = []
recall = []
cluster_distance = ['euclidean_churn','chebyshev_churn','sqeuclidean_churn']
for i in cluster_distance:
    roc_aucs.append(roc_auc_score(clean_df['naive_churn'], clean_df[i]))
    accuracy.append(accuracy_score(clean_df['naive_churn'], clean_df[i]))
    recall.append(recall_score(clean_df['naive_churn'], clean_df[i]))
scores = pd.DataFrame(columns = ['silhouette','roc_auc','accuracy','recall'])
scores['silhouette'] = silhouettes
scores['roc_auc'] = roc_aucs
scores['accuracy'] = accuracy
scores['recall'] = recall
scores.index = cluster_distance
scores

,silhouette,roc_auc,accuracy,recall
euclidean_churn,0.710371,0.653257,0.318267,1.0
chebyshev_churn,-0.631262,0.499042,0.981168,0.0
sqeuclidean_churn,0.684860,0.680077,0.370998,1.0


In [731]:
# исходя из свода метрик наилучшей выглядит разбивка по sqeuclidean_churn

In [732]:
clean_df[clean_df['sqeuclidean_churn'] == 1].shape

(343, 27)

In [733]:
if clean_df[clean_df['sqeuclidean_churn'] == 1].shape[0]>clean_df[clean_df['sqeuclidean_churn'] == 0].shape[0]:
    clean_df['best_cluster_churn'] = clean_df['sqeuclidean_churn'].apply(lambda x : 1 if x == 0 else 0)
    clean_df[clean_df['best_cluster_churn'] == 1].shape

In [734]:
clean_df.to_csv('part_2.csv')

In [735]:
feature_df.to_csv('part_2_feature_df.csv')


In [736]:
feature_df

,able,able get,able help,able redeem,able watch,absolutely,absolutely horrendous,absolutely suck,acce,access,...,zip code,𝐋𝐄𝐀𝐃𝐄𝐑,location_proba,rfm,sin_time,cos_time,text_len,topic,fact,polarity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.003766,331,-0.002982,0.999996,51,4,1,-0.150000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.425612,331,-0.013453,0.999910,52,12,1,-0.125000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.425612,333,-0.019634,0.999807,63,13,0,0.400000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.425612,333,-0.020070,0.999799,50,11,1,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.425612,333,-0.024287,0.999705,71,11,0,0.450000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.425612,333,-0.337369,0.941372,56,11,1,0.100000
527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.001883,333,-0.338669,0.940905,25,16,1,0.000000
528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.105461,333,-0.341952,0.939717,50,9,0,0.411111
529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.105461,333,-0.344616,0.938744,52,4,1,0.000000


### вопросы:

    1. нужно ли нормализовать данные перед отправкой на кластеризацию - с ней качество кластеризации падает
    2. нормальная ли логика проверки адекватности кластеризации - проверить по silhouette и сравнить roc_aucом с наивными лейблами 
    3. можно ли сказать,что часть кода (алгоритм) была заинмствована 
    4. rule-based не работает , потому что он накидывался на ручные лейблы

### как продолжать работу:

    1.upd_EDA project 

       - унифицировать картинки + отсортировать по убыванию
       - ??? проработать emoji
       - поиск твитов по неделям
    
    2. clustering 
        
       V кумулятивный tfidf - поставил критерий max_feature
       V  добавить text_len и topic ,fact,sin_time	cos_time в кластериазцию
       V добавить метрики кластеризации
                попробовать разбиения с разными расстояниями и вывести лучшее по silhette score
                сравнить разбиения с разными расстояниями против размеченных данных вручную
       ?V нормировать значения перед модельками
       V причины оттока - файл Артема
       
    
    3. rule-based
        
        V добавить свои слова в поиск 10 слов
        V таргетировать churn по rule_based alg 
        V сохранить новый churn в clean_df и в файл
        
     4. classification
        V почитать про наивный байес
        V заменить модель на lightgbm 
         
     5. текст + презентация
         - текст + картинки + презентация (оставить 2-3 дня на это)